In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image, ImageDraw
import os
import pandas as pd
import tqdm

from torchvision.transforms import v2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class CustomFaceNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomFaceNet, self).__init__()
        self.facenet = InceptionResnetV1(pretrained='vggface2', classify=False).eval()

        for param in self.facenet.parameters():
            param.requires_grad = False

        self.logits = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.facenet(x)
        x = self.logits(x)
        return x


In [11]:
transforms = v2.Compose([
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(15),
    v2.RandomResizedCrop(160, scale=(0.8, 1.0)),
    v2.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    v2.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    v2.RandomGrayscale(p=0.1),
    v2.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    v2.Resize(160),
    v2.ToTensor(),
])

c:\Users\thanh\AppData\Local\anaconda3\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [12]:
from torchvision import datasets

dataset = datasets.ImageFolder('Face Dataset/Train', transform=transforms)
dataloader = DataLoader(dataset, shuffle=True)
num_classes = len(dataset.classes)


In [13]:
# model = CustomFaceNet(num_classes)

# print(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# for epoch in range(100):
#     running_loss = 0.0
#     for i, (images, labels) in enumerate(dataloader):
#         images = images.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f'Epoch: {epoch}, Loss: {running_loss/len(dataloader)}')
# torch.save(model.state_dict(), 'facenet.pth')

In [14]:
from IPython import display

In [22]:
mtcnn = MTCNN(keep_all=True, device=device)
model = CustomFaceNet(num_classes)
model.load_state_dict(torch.load('facenet.pth'))
model.to(device)

def draw_boxes(image, boxes):
    draw = ImageDraw.Draw(image)
    for box in boxes:
        draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    return image

def get_faces(image):
    boxes, _ = mtcnn.detect(image)
    if boxes is not None:
        faces = [image.crop(box) for box in boxes]
        return faces, boxes
    return None, None

def get_embedding(face, transforms):
    face = transforms(face).unsqueeze(0).to(device)
    embedding = model(face)
    return embedding

def get_prediction(embedding):
    with torch.no_grad():
        prediction = torch.argmax(embedding, dim=1).item()
    return prediction

def get_name(prediction):
    return dataset.classes[prediction]

def recognize(image):
    faces, boxes = get_faces(image)
    if faces is not None:
        for face, box in zip(faces, boxes):
            embedding = get_embedding(face, transforms)
            prediction = get_prediction(embedding)
            name = get_name(prediction)
            image = draw_boxes(image, [box])
            draw = ImageDraw.Draw(image)
            draw.text((box[0], box[1]), name, font=None, fill=(255, 0, 0))
    return image

image = Image.open(r'E:\Project\Face Verification\Face Dataset\Test\Tran Quang Thanh\c25a6226-1e3a-4316-ad27-2393cc4a9ecc.jfif')
image = recognize(image)
display.display(image)


FileNotFoundError: [Errno 2] No such file or directory: 'facenet.pth'